In [105]:
# IMPORTANT POINTS TO REMEMBER WHILE RUNNING THE CODE
# Update with the path to the folder containing the text files
# Mention the Path to the output CSV file

In [106]:
import numpy as np
import pandas as pd

Parsing Company Name, Ticker Symbol, Participants and Answer

In [107]:
import re

def extract_text_end(text, start_word):
    pattern = re.escape(start_word) + r'(.*)'
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    else:
        return None


In [108]:
import re

def extract_text_between(text, start_word, end_word):
    if not end_word:
        pattern = re.escape(start_word) + r'(.*)'
    else:
        pattern = re.escape(start_word) + r'((?:\s+\S+){0,10000})\s+' + re.escape(end_word)

    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    if match:
        extracted_text = match.group(1).strip()
        return extracted_text
    else:
        return None

In [109]:
#REMOVING ALL THE SLIDES,PRESENTATIONS,WEBCASTS,ANNUAL REPORT, CONFERENCES

In [110]:
import os
import glob

directory = '../SeekingAlpha/article'  # Specify the directory where the files are located

keywords = ['slideshow', 'webcast', 'slides', 'presentation','conference','investor-day','annual']

for keyword in keywords:
    pattern = os.path.join(directory, f'*{keyword}*')
    files = glob.glob(pattern, recursive=True)
    
    for file_path in files:
        os.remove(file_path)
#         print(f"Deleted file: {file_path}")


In [111]:
#CREATING OPENING AND QNA FILES RENAMED AS TICKER_Q_YEAR

In [112]:
import os
import re
import textwrap
log=[]
folder_path = '../SeekingAlpha/article'
openings_folder = 'openings'
qna_folder = 'qna'

if not os.path.exists(openings_folder):
    os.makedirs(openings_folder)

if not os.path.exists(qna_folder):
    os.makedirs(qna_folder)

for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        filenamee=file_name
        with open(file_path, 'r') as text_file:
            text = text_file.read()
        
        text = ' '.join(text.strip().lower().split())
        company_match = re.search(r'(.+?) \((.+?)\)', text)
        company_name = company_match.group(1) if company_match else ''
#         ticker_symbol = company_match.group(2) if company_match else ''
        
        pattern = r"(?<=[:/])[a-z]+"
        match = re.search(pattern, text)
        if match:
            ticker_symbol = match.group()

        pattern = r"\b\d{4}\b"
        match = re.search(pattern, text)
        if match:
            year = match.group()
        pattern = r"q\d"
        match = re.search(pattern, text)
        if match:
            quarter = match.group()
          
        

        operator_text = extract_text_end(text, 'question-and-answer')
        
        text = ' '.join(text.strip().lower().split())
        company_says = extract_text_between(text, 'operator', 'question-and-answer session')
        if not company_says:
            company_says = extract_text_between(text, 'operator', 'questions')
        if not company_says:
            company_says = extract_text_between(text, 'operator', 'question')
        if not company_says:
            company_says = extract_text_between(text, 'operator', '')
            

        file_name = f'{ticker_symbol}_{year}_{quarter}.txt'

        openings_file_path = os.path.join(openings_folder, file_name)
        if company_says is None:
            log.append({filenamee})
        if company_says is not None:
            with open(openings_file_path, 'w') as openings_file:
                openings_file.write(company_says)

        qna_file_path = os.path.join(qna_folder, file_name)
        if operator_text is not None:
            with open(qna_file_path, 'w') as qna_file:
                qna_file.write(operator_text)

print("Files generated successfully!")


Files generated successfully!


In [113]:
import os  # Import the os module for file operations
import re  # Import the re module for regular expressions
import csv  # Import the csv module for CSV file operations
import textwrap  # Import the textwrap module for text formatting

folder_path = '../SeekingAlpha/article'  # Update with the path to the folder containing the text files
csv_file_path = 'parsed_data_all.csv'  # Path to the output CSV file

# Initialize the CSV file and write the header row
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Ticker Symbol', 'Year', 'Quarter','Company Participants', 'Conference Participants',
                     'Answer', 'Openings'])

    # Iterate over the files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):  # Process only .txt files
            file_path = os.path.join(folder_path, file_name)

            # Read the text file
            with open(file_path, 'r') as text_file:
                text = text_file.read()

            text = ' '.join(text.strip().lower().split())
            company_match = re.search(r'(.+?) \((.+?)\)', text)
#             company_name = company_match.group(1) if company_match else ''
    #         ticker_symbol = company_match.group(2) if company_match else ''

            pattern = r"(?<=[:/])[a-z]+"
            match = re.search(pattern, text)
            if match:
                ticker_symbol = match.group()

            pattern = r"\b\d{4}\b"
            match = re.search(pattern, text)
            if match:
                year = match.group()
            pattern = r"q\d"
            match = re.search(pattern, text)
            if match:
                quarter = match.group()



            operator_text = extract_text_end(text, 'question-and-answer')

            text = ' '.join(text.strip().lower().split())
            company_says = extract_text_between(text, 'operator', 'question-and-answer session')
            if not company_says:
                company_says = extract_text_between(text, 'operator', 'questions')
            if not company_says:
                company_says = extract_text_between(text, 'operator', 'question')
            if not company_says:
                company_says = extract_text_between(text, 'operator', '')

            # Write the extracted information to the CSV file
            writer.writerow([ticker_symbol, year,quarter, ', '.join(company_participants),
                             ', '.join(conference_participants), operator_text, company_says])

print("CSV file generated successfully!")


CSV file generated successfully!


In [114]:
df=pd.read_csv('parsed_data_all.csv')

In [115]:
df

,Ticker Symbol,Year,Quarter,Company Participants,Conference Participants,Answer,Openings
0,dhiff,2016,q2,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session operator [operator instructions]. your...,welcome to the dh corporation conference call ...
1,zagg,2020,q1,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session. our first quarter earnings press rele...,"good afternoon, ladies and gentlemen, and welc..."
2,ora,2011,q4,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session operator [operator instructions] your ...,"ladies and gentlemen, thank you for standing b..."
3,ora,2007,q4,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session towards the end of this conference. (o...,"good day, ladies and gentlemen, and welcome to..."
4,ora,2011,q2,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session operator [operator instructions] your ...,"good morning. my name is julianne, and i will ..."
...,...,...,...,...,...,...,...
432,air,2022,q1,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session operator [operator instructions] speak...,"good afternoon, ladies and gentlemen, welcome ..."
433,ora,2009,q3,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session operator (operator instructions). the ...,at this time i would like to welcome everyone ...
434,ora,2019,q3,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session operator [operator instructions]. the ...,good morning and welcome to the ormat technolo...
435,ora,2021,q3,"Marybeth Csaby - IR, KCSA Strategic Communicat...","Lasan Johong - RBC Capital Market, Gregg Orril...",session operator [operator instructions] our f...,"hello, and welcome to the ormat technologies q..."
